In [33]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm

In [34]:
df = pd.read_parquet('../../../metrics.parquet')

In [35]:
df, _ = train_test_split(df, train_size=200000, stratify=df['time_to_stop_activity'], random_state=11)

In [36]:
df = df.drop(
    columns = [
        "user", 
        'project', 
        'current_month', 
        'turnover_num',
        'turnover',
        'betweenness_centrality_12_intercept',
        'betweenness_centrality_12_slope', 
        'betweenness_centrality_12_std_dev',
        'betweenness_centrality_3_intercept', 
        'betweenness_centrality_3_slope',
        'betweenness_centrality_3_std_dev',
        'betweenness_centrality_6_intercept', 
        'betweenness_centrality_6_slope',
        'betweenness_centrality_6_std_dev',
        'betweenness_centrality_9_intercept', 
        'betweenness_centrality_9_slope',
        'betweenness_centrality_9_std_dev'
    ], axis = 1
)

In [37]:
df = df.astype(np.float64)
df = df.replace([np.inf], np.nan)

for column in df.columns:
    max_value = df[column].max(skipna=True) 
    df[column] = df[column].fillna(max_value)

df = df.replace([-np.inf], np.nan)

for column in df.columns:
    max_value = df[column].min(skipna=True) 
    df[column] = df[column].fillna(max_value)

In [38]:
for column in df.columns:
    max_value = df[column].max(skipna=True) 
    df[column] = df[column].fillna(max_value)

In [39]:
def train_evaluate_adaboost(n_estimators, learning_rate, max_depth, X_train, Y_train, X_test, Y_test):
    mse = None; mae = None; r2 = None

    try:
        base_estimator = DecisionTreeRegressor(max_depth=max_depth)
        adaboost = AdaBoostRegressor(
            estimator=base_estimator,
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            random_state=11
        )
        
        adaboost.fit(X_train, Y_train)
        y_pred = adaboost.predict(X_test)

        mse = mean_squared_error(Y_test, y_pred)
        mae = mean_absolute_error(Y_test, y_pred)
        r2 = r2_score(Y_test, y_pred)

        print(f'AB - Estimators: {n_estimators}, Learning Rate: {learning_rate}, Max Depth: {max_depth} Finalized - {mse}, {mae}, {r2}')
    
    except Exception as e:
        print(f'AB - Estimators: {n_estimators}, Learning Rate: {learning_rate}, Max Depth: {max_depth} Error: {e}')
        
    return {
        'n_estimators': n_estimators,
        'learning_rate': learning_rate,
        'max_depth': max_depth, 
        'mean_squared_error': mse,
        'mean_absolute_error': mae,
        'r2_score': r2
    }

In [40]:
param_combinations = [
    (n_estimators, learning_rate, max_depth) 
    for n_estimators in [50, 100, 200]
    for learning_rate in [0.01, 0.1, 1.0]
    for max_depth in [3, 5, 10]
]


In [41]:
x = df.drop(columns = ['time_to_stop_activity'], axis = 1).values
y = df['time_to_stop_activity'].values

In [42]:
X_train, X_test, Y_train, Y_test = train_test_split(
    x, y, 
    test_size = 0.3, 
    random_state=11
)

In [43]:
benchmark = Parallel(n_jobs=-1)(
    delayed(train_evaluate_adaboost)(n_estimators, lr, depth, X_train, Y_train, X_test, Y_test)
    for n_estimators, lr, depth in param_combinations
)

AB - Estimators: 100, Learning Rate: 1.0, Max Depth: 3 Finalized - 7.981074181366234, 1.972236686274031, 0.5398922850129593
AB - Estimators: 50, Learning Rate: 1.0, Max Depth: 3 Finalized - 7.981074181366234, 1.972236686274031, 0.5398922850129593
AB - Estimators: 50, Learning Rate: 1.0, Max Depth: 5 Finalized - 7.400968229962559, 1.8444632868849884, 0.5733353050482695
AB - Estimators: 100, Learning Rate: 1.0, Max Depth: 5 Finalized - 7.400968229962559, 1.8444632868849884, 0.5733353050482695
AB - Estimators: 50, Learning Rate: 0.01, Max Depth: 3 Finalized - 7.013364219528288, 1.5638581051755807, 0.5956806174094931


/Users/pro32/Library/Python/3.9/lib/python/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


AB - Estimators: 50, Learning Rate: 0.1, Max Depth: 3 Finalized - 7.698710535312938, 1.8992394530514154, 0.5561705063436564
AB - Estimators: 100, Learning Rate: 0.1, Max Depth: 3 Finalized - 7.698710535312938, 1.8992394530514154, 0.5561705063436564
AB - Estimators: 50, Learning Rate: 0.01, Max Depth: 5 Finalized - 6.279452729270897, 1.3245869056393262, 0.6379905034112363
AB - Estimators: 50, Learning Rate: 0.1, Max Depth: 5 Finalized - 7.006122072576565, 1.757497274592305, 0.5960981260818642
AB - Estimators: 200, Learning Rate: 1.0, Max Depth: 3 Finalized - 7.981074181366234, 1.972236686274031, 0.5398922850129593
AB - Estimators: 200, Learning Rate: 0.1, Max Depth: 3 Finalized - 7.698710535312938, 1.8992394530514154, 0.5561705063436564
AB - Estimators: 200, Learning Rate: 1.0, Max Depth: 5 Finalized - 7.400968229962559, 1.8444632868849884, 0.5733353050482695
AB - Estimators: 100, Learning Rate: 0.01, Max Depth: 3 Finalized - 7.0439716212250625, 1.6150139694640828, 0.5939161053480374
AB

In [44]:
benchmark_df = pd.DataFrame(benchmark)

In [45]:
benchmark_df.to_excel('AdaBoostRegressorBenchmark.xlsx')